<a href="https://colab.research.google.com/github/hersan19/EMC_2021/blob/main/Ejemplo_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deap

     |████████████████████████████████| 160 kB 15.5 MB/s 


In [ ]:
import array
import random
import json
import numpy
from deap import algorithms,base,creator,tools
import pandas as pd
import numpy as np

In [ ]:

    
import math
def Euclidian(X, Y):
    return math.sqrt( (float(X[1])-float(Y[1]))**2 + (float(X[0])-float(Y[0]))**2)
    
def Distances(cities):
    r= len(cities)
    M=np.zeros((r,r))
    for i in range(r-1):
        for j in range(i+1, r):
            M[i][j] = Euclidian(cities[i], cities[j])
            M[j][i]= M[i][j]
    return M


url="https://raw.githubusercontent.com/hersan19/EMC_2021/main/ciudades.csv?token=AMUQEUUNCZHPLVM2N6WHX5DBBKZ4S"
df_ciudades=pd.read_csv(url)
datas = list(zip(list(df_ciudades.latitud), list(df_ciudades.longitud)))

distance_map =Distances(datas)
IND_SIZE = len(datas)

  


creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalTSP(individual):
    distance = distance_map[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2]
    return distance,

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalTSP)




def main():
    #random.seed(0)

    pop = toolbox.population(n=300)
    #print(pop[0])
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 1000, stats=stats, 
                        halloffame=hof)
    
    return pop, stats,hof

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [ ]:
pop,estadisiticas,mejor=main()

gen	nevals	avg    	std    	min    	max    
0  	300   	262.048	18.8844	209.047	300.588
1  	227   	251.047	17.5819	200.657	304.05 
2  	238   	247.431	20.2122	199.866	314.862
3  	234   	242.522	20.448 	191.01 	298.548
4  	231   	236.6  	20.2554	181.515	306.892
5  	233   	236.779	22.4793	173.164	293.073
6  	234   	235.186	24.3104	172.883	291.325
7  	219   	230.668	24.0633	172.883	293.457
8  	222   	227.837	24.9969	176.37 	297.165
9  	238   	227.413	25.8216	155.496	305.956
10 	220   	223.406	24.4744	158.402	289.075
11 	230   	220.02 	25.2874	156.87 	294.813
12 	232   	222.406	27.9049	156.87 	298.863
13 	225   	221.417	27.6406	150.849	293.375
14 	240   	221.113	26.415 	156.87 	294.089
15 	234   	220.137	28.5449	156.057	293.297
16 	225   	214.465	27.4024	156.057	299.903
17 	233   	213.611	27.0805	154.202	283.368
18 	234   	212.566	29.2965	152.864	289.389
19 	215   	207.714	29.4204	149.443	278.874
20 	216   	199.287	30.0017	148.265	283.245
21 	226   	187.695	23.5236	148.265	277.909
22 	226   	

In [ ]:
mejor[0][:]


array('i', [29, 19, 5, 26, 22, 30, 3, 10, 7, 13, 17, 4, 25, 1, 2, 24, 8, 31, 0, 21, 11, 12, 27, 18, 6, 23, 9, 15, 14, 16, 20, 28])

In [ ]:
df_ciudades

In [ ]:
mejor[0]



Individual('i', [10, 19, 5, 26, 22, 30, 3, 15, 7, 13, 17, 4, 25, 1, 2, 24, 8, 31, 0, 14, 16, 11, 27, 6, 18, 23, 9, 21, 12, 29, 28, 20])

In [ ]:
recorrido=[]
for i in range(32):
  latitud=df_ciudades.iloc[mejor[0][i]]["latitud"]
  longitud=df_ciudades.iloc[mejor[0][i]]["longitud"]
  recorrido.append((latitud,longitud))

In [ ]:
recorrido

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig=go.Figure()
fig.add_trace(go.Scattergeo(
    #locations = ["Mexico"],
    #locationmode = 'country names',
    lon=df_ciudades["longitud"],
    lat=df_ciudades["latitud"],
    hoverinfo="text",
    text=df_ciudades["ciudad"],
    mode="markers",
    marker = dict(size=2,color='rgb(255,0,0)',line=dict(width=3,color='rgba(68,68,0)'))))

for i in range(31):
  fig.add_trace(go.Scattergeo(
      lat=[recorrido[i][0],recorrido[i+1][0]],
      lon=[recorrido[i][1],recorrido[i+1][1]],
      mode="lines",
      line=dict(width=1,color="blue"),
  ))
fig.add_trace(go.Scattergeo(
    lat=[recorrido[-1][0],recorrido[0][0]],
    lon=[recorrido[-1][1],recorrido[0][1]],
    mode="lines",
    line=dict(width=1,color="blue"),
  ))

fig.update_layout(title_text="Mejor recorrido",showlegend=False,
                  geo =dict(
                      #showcountries=True,
                      scope="north america",
                      projection_type ='conic equal area',                      
                      showland = True,
                      #landcolor='rgb(243,243,243)',
                      #countrycolor='rgb(204,204,204)',
                  ), width=1800, height=1400)
fig.show()